In [1]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn

In [2]:
import logging
import os
import evaluate
import numpy as np
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DebertaV2Config,
    DebertaV2Model,
    TrainingArguments, 
    Trainer, 
    DataCollatorWithPadding
)
from transformers.models.deberta_v2.modeling_deberta_v2 import DebertaV2Encoder


2025-05-16 07:19:01.301207: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747379941.503927      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747379941.563797      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(sys.stdout)
    ],
    force=True
)

logger = logging.getLogger(__name__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

os.makedirs("./result", exist_ok=True)
os.makedirs("./logs", exist_ok=True)
os.makedirs("./checkpoint", exist_ok=True) 


Using device: cuda


In [4]:
imdb_dataset = load_dataset("imdb")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
teacher_model_id = 'microsoft/deberta-v3-base'
tokenizer = AutoTokenizer.from_pretrained(teacher_model_id)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [6]:

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512) 


In [7]:
tokenized_datasets = imdb_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

accuracy_metric = evaluate.load("accuracy") 

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)


In [9]:
tokenized_train = tokenized_datasets["train"] 
split = tokenized_datasets["test"].train_test_split(test_size=0.8)
tokenized_val = split["train"]  # 20%
tokenized_test = split["test"]  # 80%

In [10]:
teacher_model_finetuned_path = '/kaggle/input/deberta-v3-base-finetuned-imdb'

In [11]:
def copy_deberta_weights(teacher, student):
    if isinstance(teacher, DebertaV2Model) or type(teacher).__name__.startswith('DebertaV2For'):
        for teacher_part, student_part in zip(teacher.children(), student.children()):
            copy_deberta_weights(teacher_part, student_part)
    elif isinstance(teacher, DebertaV2Encoder):
        teacher_encoding_layers = [layer for layer in next(teacher.children())]
        student_encoding_layers = [layer for layer in next(student.children())]
        for i in range(len(student_encoding_layers)):
            student_encoding_layers[i].load_state_dict(teacher_encoding_layers[2*i].state_dict())
    else:
        student.load_state_dict(teacher.state_dict())

def create_student(teacher_model):
    configuration = teacher_model.config.to_dict()
    original_num_layers = configuration["num_hidden_layers"]
    configuration["num_hidden_layers"] //= 2
    logger.info(f"Teacher layers: {original_num_layers}, Student layers: {configuration['num_hidden_layers']}")
    student_config = DebertaV2Config.from_dict(configuration)
    student_config.num_labels = teacher_model.config.num_labels
    student_model = type(teacher_model)(config=student_config)
    copy_deberta_weights(teacher_model, student_model)
    return student_model

In [12]:
teacher_model_distill = AutoModelForSequenceClassification.from_pretrained(teacher_model_finetuned_path, num_labels=2)
student_model = create_student(teacher_model_distill)
student_model_init_path = 'deberta-v3-student-init-imdb'
student_model.save_pretrained(student_model_init_path)
tokenizer.save_pretrained(student_model_init_path)

2025-05-16 07:20:37,276 [INFO] Teacher layers: 12, Student layers: 6


('deberta-v3-student-init-imdb/tokenizer_config.json',
 'deberta-v3-student-init-imdb/special_tokens_map.json',
 'deberta-v3-student-init-imdb/spm.model',
 'deberta-v3-student-init-imdb/added_tokens.json',
 'deberta-v3-student-init-imdb/tokenizer.json')

In [13]:
del teacher_model_distill
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [14]:
class DistillationTrainingArguments(TrainingArguments):
    def __init__(self, *args, alpha=0.5, temperature=2.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.alpha = alpha
        self.temperature = temperature

class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher = teacher_model
        self._move_model_to_device(self.teacher, self.model.device)
        self.teacher.eval()

    def compute_loss(self, model, inputs, return_outputs=False,num_items_in_batch=None):
        outputs_student = model(**inputs)
        student_loss = outputs_student.loss
        logits_student = outputs_student.logits

        with torch.no_grad():
            outputs_teacher = self.teacher(**inputs)
        logits_teacher = outputs_teacher.logits

        temperature = self.args.temperature
        alpha = self.args.alpha

        loss_fct = nn.KLDivLoss(reduction="batchmean")
        distillation_loss = loss_fct(
            F.log_softmax(logits_student / temperature, dim=-1),
            F.softmax(logits_teacher / temperature, dim=-1)
        ) * (temperature ** 2)

        loss = alpha * student_loss + (1.0 - alpha) * distillation_loss
        return (loss, outputs_student) if return_outputs else loss


In [15]:
teacher_model_for_distill = AutoModelForSequenceClassification.from_pretrained(
    teacher_model_finetuned_path, num_labels=2
).to(device) 

student_model_for_distill = AutoModelForSequenceClassification.from_pretrained(
    student_model_init_path, num_labels=2
).to(device)


In [16]:
tokenizer = AutoTokenizer.from_pretrained(student_model_init_path) # 从学生或教师目录加载都行

distill_output_dir = "./distill_checkpoints"
distill_logging_dir = './distill_logs'

distillation_args = DistillationTrainingArguments(
    output_dir=distill_output_dir,
    warmup_ratio=0.1,                   
    num_train_epochs=3,                 
    per_device_train_batch_size=8,    
    per_device_eval_batch_size=16,    
    gradient_accumulation_steps=2,
    learning_rate=3e-5,                
    weight_decay=0.01,
    logging_dir=distill_logging_dir,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",            
    load_best_model_at_end=True,      
    metric_for_best_model="accuracy",
    fp16=torch.cuda.is_available(),
    report_to="tensorboard",
    alpha=0.5,                         
    temperature=4.0      
)


In [17]:
distill_trainer = DistillationTrainer(
    model=student_model_for_distill,   
    teacher_model=teacher_model_for_distill, 
    args=distillation_args,            
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,         
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_19/645987027.py:9: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `DistillationTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


In [18]:
distill_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.494200,0.427762,0.872400
2,0.268100,0.397699,0.891000


TrainOutput(global_step=4686, training_loss=0.44505704461739604, metrics={'train_runtime': 5437.8669, 'train_samples_per_second': 13.792, 'train_steps_per_second': 0.862, 'total_flos': 9930109908910080.0, 'train_loss': 0.44505704461739604, 'epoch': 2.9984})

In [19]:

final_student_model_path = "deberta-v3-student-distilled-imdb"
distill_trainer.save_model(final_student_model_path)
tokenizer.save_pretrained(final_student_model_path)


('deberta-v3-student-distilled-imdb/tokenizer_config.json',
 'deberta-v3-student-distilled-imdb/special_tokens_map.json',
 'deberta-v3-student-distilled-imdb/spm.model',
 'deberta-v3-student-distilled-imdb/added_tokens.json',
 'deberta-v3-student-distilled-imdb/tokenizer.json')

In [20]:
del teacher_model_for_distill
del student_model_for_distill
del distill_trainer
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [21]:
final_student_model = AutoModelForSequenceClassification.from_pretrained(
    final_student_model_path, num_labels=2
).to(device) 


eval_args = TrainingArguments(
    output_dir='./eval_output',
    per_device_eval_batch_size=32, 
    do_train=False, 
    do_eval=True, 
    report_to="none"
)

eval_trainer = Trainer(
    model=final_student_model,
    args=eval_args,
    eval_dataset=tokenized_test, 
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_19/3141225457.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  eval_trainer = Trainer(


In [22]:
evaluation_results = eval_trainer.evaluate()
print(evaluation_results)

{'eval_loss': 0.3296298086643219, 'eval_model_preparation_time': 0.0018, 'eval_accuracy': 0.89575, 'eval_runtime': 255.3827, 'eval_samples_per_second': 78.314, 'eval_steps_per_second': 2.447}
